## import

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import pandas as pd

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../

# load data

In [2]:
import os
import shutil

# 데이터셋 경로 설정하기
dataset_path = './usg1_dataset/train_images/'
dir = "./usg1_dataset/"
image_dir = dir+'train_images/'
train_csv_dir = dir+'train.csv'
train_copy_dir = './usg1_dataset/train_copy/'
test_dir = './usg1_dataset/test_images/'
split_label_dir = dir + 'train_split_label/'

In [3]:
os.makedirs(split_label_dir, exist_ok=True)

train_dir = split_label_dir + '/train/'
valid_dir = split_label_dir + '/valid/'
test_dir = split_label_dir + '/test/'

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


shutil.rmtree(train_dir)
shutil.rmtree(valid_dir)
shutil.rmtree(test_dir)

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


# 레이블 이름 설정하기
train_labels = pd.read_csv(train_csv_dir)
labels = train_labels['label'].to_list() # 레이블 리스트
labels_str = []
for label in labels:
    labels_str.append(str(label))

# 하위 폴더 생성하기
for label in labels_str:
    os.makedirs(train_dir + label, exist_ok=True)
    os.makedirs(valid_dir + label, exist_ok=True)

In [4]:
import os
from natsort import natsorted

# 파일 이름 변경하기 ""

# 1) copy : train_images => train_copy (and test)

# train
#os.makedirs(train_copy_dir)
shutil.rmtree(train_copy_dir)
os.makedirs(train_copy_dir)

filenames = natsorted(os.listdir(image_dir))
for filename in filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(image_dir, filename)
        shutil.copy(name, train_copy_dir)


# test
test_dir = './usg1_dataset/test_images/'
split_test_dir = split_label_dir + 'test/'

test_filenames = natsorted(os.listdir(test_dir))

for filename in test_filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(test_dir, filename)
        shutil.copy(name, split_test_dir)

In [5]:
filenames = natsorted(os.listdir(train_copy_dir))

# 2) rename
for idx, filename in enumerate(filenames):
        label = labels_str[idx]

        name = os.path.join(train_copy_dir, filename)
        labeled_name = os.path.join(train_copy_dir, os.path.splitext(filename)[0]+ '_' +label+'.jpg')
        os.rename(name, labeled_name)

In [6]:
# 2) split 
from sklearn.model_selection import train_test_split

filenames = natsorted(os.listdir(train_copy_dir))

train_filenames, valid_filenames = train_test_split(filenames, test_size=0.1, random_state=42)

train_filenames = natsorted(train_filenames)
valid_filenames = natsorted(valid_filenames)

print('train\n',train_filenames[:5])
print('valid\n',valid_filenames[:5])

train
 ['000000_1.jpg', '000001_8.jpg', '000002_0.jpg', '000003_1.jpg', '000004_8.jpg']
valid
 ['000008_9.jpg', '000014_1.jpg', '000019_0.jpg', '000031_0.jpg', '000033_6.jpg']


In [7]:
# 이미지 파일 이동하기

# train
for file in train_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, train_dir + label + '/' + file)

# valid
for file in valid_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, valid_dir + label + '/' + file)

# Model

In [8]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

device = torch.device('mps')

model = vgg16(pretrained=True) # ❶ vgg16 모델 객체 생성
fc = nn.Sequential( # ❷ 분류층의 정의
       nn.Linear(512 * 7 * 7, 4096),
       nn.ReLU(),
       nn.Dropout(), #❷ 드롭아웃층 정의
       nn.Linear(4096, 4096),
       nn.ReLU(),
       nn.Dropout(),
       nn.Linear(4096, 10),
   )

model.classifier = fc # ➍ VGG의 classifier를 덮어씀
model.to(device)

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
import tqdm

from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.optim.adam import Adam

transforms = Compose([
   Resize(224),
   RandomCrop((224, 224), padding=4),
   RandomHorizontalFlip(p=0.5),
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

batch size = 16

In [10]:
# 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir,
                                      transform=transforms)

valid_dataset = datasets.ImageFolder(root=valid_dir,
                                      transform=transforms)

'''test_dataset = datasets.ImageFolder(root=split_test_dir,
                                      transform=transforms)'''


# 데이터로더 생성하기
batch_size = 16
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

'''test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)'''

'test_loader = torch.utils.data.DataLoader(dataset=test_dataset,\n                                           batch_size=batch_size,\n                                           shuffle=True)'

In [11]:
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

for epoch in range(10):
   iterator = tqdm.tqdm(train_loader) # ➊ 학습 로그 출력
   for data, label in iterator:
       optim.zero_grad()

       preds = model(data.to(device)) # 모델의 예측값 출력

       loss = nn.CrossEntropyLoss()(preds, label.to(device))
       loss.backward()
       optim.step()
     
       # ❷ tqdm이 출력할 문자열
       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "0309_model_VGG.pth") # 모델 저장

epoch:10 loss:0.31659990549087524: 100%|██████████| 716/716 [02:48<00:00,  4.25it/s]   


In [12]:
model.load_state_dict(torch.load("0309_model_VGG.pth", map_location=device))

num_corr = 0

with torch.no_grad():
   for data, label in valid_loader:

       output = model(data.to(device))
       preds = output.data.max(1)[1]
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(valid_dataset)}")

Accuracy:0.9323899371069182
